# ROUGE
Recall-Oriented Understudy for Gisting Evaluation

Before we start, there are about fifty gazzilion variations of ROUGE

However, all ROUGE-s rely on recall, precision, and F1 score

## Theory: Basics
> $
> \text{Precision} = \dfrac{\text{Relevant retrieved instances}}{\text{All }\textbf{retrieved}\text{ instances}}
> $
> 
> "I found 5 pages with text relevant to the query out of 7 total pages"
> 
> "I hit 5 targets on the dot out of 7 total targets"

> $
> \text{Recall} = \dfrac{\text{Relevant retrieved instances}}{\text{All }\textbf{relevant}\text{ instances}}
> $
> "I recalled 5 memories out of 7 total **relevant** memories"

Both recall and precision divide **relevant retrieved** instances. The nominator is always the True Positive. The only difference is in the denominator.

> $
> F1 = \dfrac{2(Precision \times Recall)}{Precision + Recall}
> $
>
> This is a balanced way to combine Precision and Recall

## ROUGE: Basics
In essence, ROUGE is a glorified F1 score that specifically computes matches between reference and candidate texts.


### ROUGE-1
We can take 2 candidate texts and 1 reference text
- Reference: "The cat sat on the mat."
- Candidate 1: "The cat sat."
- Candidate 2: "The cat sat on the mat, and the dog barked at the moon, and the cow jumped over the sun."

Candidate 1:

> **Precision:**  3 / 3 = 1.0. All 3 of the words used were in the reference.
> 
> **Recall:** 3 / 6 = 0.5. 3 out of the 6 words were captured from the reference.

Now, for candidate 2:

> **Precision:** 6 / 21 ≈ 0.29. Only 6 out of 21 total words were relevant.
>
> **Recall:** 6 / 6 = 1.0. 6 out of the 6 words were captured from the reference.

We can obviously see that F1 score will be much lower for candidate 2 compared to candidate 1. Candidate 1 is better. This makes sense, since the first candidate capture most of the relevant meaning without adding extra.

### ROUGE-2 & ROUGE-N
ROUGE-1 has an issue with this case:

> **Reference:** "The president signed the new bill."
> 
> **Candidate:** "The bill signed the new president."

ROUGE-1 would say that the candidate reference is ideal complement for the reference. But the meaning is completely lost.

An improvement, ROUGE-2 (and ROUGE-N by extension) uses the same idea, but splits the sentence into bigrams (or n-grams respectively). Now:

> **Reference Bigrams:** "the president", "president signed", "signed the", "the new", "new bill"
>
> **Candidate Bigrams:** "the bill", "bill signed", "signed the", "the new", "new president"

Now, F1 score would obviously show that the candidate summary is actually bad for the given reference.

A generalization is ROUGE-N. The larger the N, the more context is captured. Note that bigger n-grams make more grams and this requires more comparisons. So, this is a trade-off

### ROUGE-L
ROUGE-N is good, but if the summary is short and good (i.e. keeps the main idea, but throws out details), ROUGE-N will penalize this behavior. Example:

> **Reference:** "The extremely quick brown fox jumps over the very lazy dog."
> 
> **Candidate:** "The quick brown fox jumps over the lazy dog."

The only difference is "extremely", and ROUGE-N will penalize this. But the candidate is still quite good. A metric that would favour this kind of cases is Longest Common Subsequence. This allows "gaps" in the candidate, as long as the candidate has the same words in the same order.

Using this metric requires modifying Precision and Recall.

> $
> \text{Recall-LCS} = \dfrac{\text{Length of LCS}}{\text{Total words in the reference summary}}
> $

Tells us what fraction of the reference summary is covered by the common subsequence.

> $
> \text{Precision-LCS} = \dfrac{\text{Length of LCS}}{\text{Total words in the candidate summary}}
> $

Tells us how much of the candidate summary is relevant and in the correct order.

Formula for F1 score is the same.

With theory out the way, let's implement all of these ROUGE-s

In [16]:
import re


def preprocess(text: str) -> list[str]:
    """
    This is your first and most important helper function.
    Before you can compare words, you need a clean, consistent list of them.
    What steps should you take to turn a raw string into a list of "tokens"?
    
    Hint: Think about case sensitivity and how to split the sentence into words.
    """
    return re.sub(r'[^A-z\s\d][\\\^]?', '', text).lower().split()

def _f1_score(precision: float, recall: float) -> float:
    """
    A helper function to calculate the F1-score.
    You already know the formula for this.
    What is one edge case you should be careful of to avoid a division error?
    """
    if precision + recall == 0:
        raise ValueError(f"Precision + recall cannot be 0, division by zero. Precision: {precision}, recall: {recall}")
    
    return 2 * precision * recall / (precision + recall)

def _create_ngrams(tokens: list[str], n: int) -> set[tuple]:
    """
    This function will be used by ROUGE-N.
    It needs to take a list of tokens and slide a window of size 'n' across it.
    
    For example, with tokens ['a', 'b', 'c', 'd'] and n=2,
    it should produce something like {('a', 'b'), ('b', 'c'), ('c', 'd')}.
    
    Why might returning a 'set' of n-grams be more efficient than a 'list'?
    """
    ngrams = set()
    for i in range(0, len(tokens) + 1 - n):
        start_ngram = i
        end_ngram = i + n
        
        ngrams.add(tuple(tokens[start_ngram : end_ngram]))
        
    return ngrams
        

def calculate_rouge_n(candidate_tokens: list[str], reference_tokens: list[str], n: int) -> dict[str, float]:
    """
    This function will calculate ROUGE-N (e.g., ROUGE-1, ROUGE-2).
    
    Here's the plan:
    1. Use your `_create_ngrams` helper to get the n-grams for both the candidate and reference.
    2. Find the number of n-grams that are present in BOTH sets. This is your overlap.
    3. Calculate precision: How many of the candidate's n-grams were relevant?
       (Hint: overlap / total number of candidate n-grams)
    4. Calculate recall: How many of the reference's n-grams were captured?
       (Hint: overlap / total number of reference n-grams)
    5. Use your `_f1_score` helper to combine precision and recall.
    6. Return all three scores in a dictionary: {"precision": p, "recall": r, "f1": f1}.
    """
    candidate_ngram = _create_ngrams(candidate_tokens, n)
    reference_ngram = _create_ngrams(reference_tokens, n)
    
    overlap = candidate_ngram.intersection(reference_ngram)
    
    p = len(overlap) / len(candidate_ngram) if len(candidate_ngram) > 0 else 0.0
    r = len(overlap) / len(reference_ngram) if len(reference_ngram) > 0 else 0.0

    f1 = _f1_score(p, r)
    
    return {"precision": p, "recall": r, "f1": f1}
    
    

def _lcs_length(X: list[str], Y: list[str]) -> int:
    """
    This is the heart of ROUGE-L. It calculates the Length of the Longest Common Subsequence.
    This is a classic dynamic programming problem.
    
    Imagine creating a grid (or a 2D array) where the rows correspond to tokens in X
    and columns correspond to tokens in Y.
    
    The value in each cell `grid[i][j]` will represent the length of the LCS
    for the subsequences `X[:i]` and `Y[:j]`.
    
    How would you fill in the value of `grid[i][j]` based on whether `X[i-1]` and `Y[j-1]` match?
    - If `X[i-1] == Y[j-1]`, it means we have found a common element. The length of the LCS
      is one more than the LCS of the subsequences before this match.
      So, `grid[i][j] = grid[i-1][j-1] + 1`.

    What do you do if they don't match?
    - If `X[i-1] != Y[j-1]`, the LCS is the longest of the two possibilities:
      1. The LCS of `X[:i]` and `Y[:j-1]` (i.e., `grid[i][j-1]`).
      2. The LCS of `X[:i-1]` and `Y[:j]` (i.e., `grid[i-1][j]`).
      So, `grid[i][j] = max(grid[i-1][j], grid[i][j-1])`.
    
    The final answer will be the number in the bottom-right corner of the grid.
    """
    m, n = len(X), len(Y)
    
    # Initialize the DP grid with zeros.
    # grid[i][j] will store the length of LCS of X[:i] and Y[:j]
    grid = [[0] * (n + 1) for _ in range(m + 1)]
    
    # Build the grid in a bottom-up fashion
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if X[i - 1] == Y[j - 1]:
                # If the current tokens match, extend the LCS from the previous diagonal cell.
                grid[i][j] = grid[i - 1][j - 1] + 1
            else:
                # If they don't match, take the maximum LCS length from the top or left cell.
                grid[i][j] = max(grid[i - 1][j], grid[i][j - 1])
                
    # The value in the bottom-right corner is the length of the LCS for the entire sequences.
    return grid[m][n]



def calculate_rouge_l(candidate_tokens: list[str], reference_tokens: list[str]) -> dict[str, float]:
    """
    This function will calculate ROUGE-L.
    
    The plan is very similar to ROUGE-N, but uses LCS instead of n-gram overlap:
    1. Use your `_lcs_lenlength` helper to find the length of the longest common subsequence.
    2. The denominators for precision and recall are simpler here. What are they?
       (Hint: think about the total number of words in each summary).
    3. Calculate Precision-LCS and Recall-LCS.
    4. Use `_f1_score` to get the final F1 score.
    5. Return the scores in a dictionary.
    """
    lcs = _lcs_length(candidate_tokens, reference_tokens)
    
    p = lcs / len(candidate_tokens)
    r = lcs / len(reference_tokens)
    
    f1 = _f1_score(p, r)
    
    return {"precision": p, "recall": r, "f1": f1}



In [17]:
# --- Example Usage ---
# Once you've implemented the functions, you can test them like this.

reference_text = "The president signed the new bill."
candidate_text = "The bill signed the new president."

# 1. Preprocess the texts
reference_tokens = preprocess(reference_text)
candidate_tokens = preprocess(candidate_text)

# 2. Calculate ROUGE-1
rouge1_scores = calculate_rouge_n(candidate_tokens, reference_tokens, 1)
print(f"ROUGE-1 Scores: {rouge1_scores}")

# 3. Calculate ROUGE-2
rouge2_scores = calculate_rouge_n(candidate_tokens, reference_tokens, 2)
print(f"ROUGE-2 Scores: {rouge2_scores}")

# 4. Calculate ROUGE-L
rougel_scores = calculate_rouge_l(candidate_tokens, reference_tokens)
print(f"ROUGE-L Scores: {rougel_scores}")


ROUGE-1 Scores: {'precision': 1.0, 'recall': 1.0, 'f1': 1.0}
ROUGE-2 Scores: {'precision': 0.4, 'recall': 0.4, 'f1': 0.4000000000000001}
ROUGE-L Scores: {'precision': 0.6666666666666666, 'recall': 0.6666666666666666, 'f1': 0.6666666666666666}
